### IMPORT LIBRARIES 

In [3]:
!pip install wfdb
!pip install peakutils
!pip install neurokit2
import neurokit2 as nk
import wfdb
from wfdb import processing
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import peakutils

### PRE-PROCESSED THE DATA 

In [2]:
# Make a temporary download directory in your current working directory
cwd = os.getcwd()
dl_dir = os.path.join(cwd, 'tmp_dl_dir')

# Download all the WFDB content
wfdb.dl_database('incartdb', dl_dir=dl_dir)

# Display the downloaded content in the folder
display(os.listdir(dl_dir))

# Cleanup: delete the downloaded directory
#shutil.rmtree(dl_dir)

Generating record list for: I01
Generating record list for: I02
Generating record list for: I03
Generating record list for: I04
Generating record list for: I05
Generating record list for: I06
Generating record list for: I07
Generating record list for: I08
Generating record list for: I09
Generating record list for: I10
Generating record list for: I11
Generating record list for: I12
Generating record list for: I13
Generating record list for: I14
Generating record list for: I15
Generating record list for: I16
Generating record list for: I17
Generating record list for: I18
Generating record list for: I19
Generating record list for: I20
Generating record list for: I21
Generating record list for: I22
Generating record list for: I23
Generating record list for: I24
Generating record list for: I25
Generating record list for: I26
Generating record list for: I27
Generating record list for: I28
Generating record list for: I29
Generating record list for: I30
Generating record list for: I31
Generati

['I66.hea',
 'I28.atr',
 'I73.atr',
 'I75.atr',
 'I43.atr',
 'I43.dat',
 'I11.atr',
 'I54.hea',
 'I64.hea',
 'I29.hea',
 'I69.atr',
 'I55.atr',
 'I53.hea',
 'I65.dat',
 'I26.dat',
 'I61.atr',
 'I40.hea',
 'I13.dat',
 'I16.atr',
 'I62.hea',
 'I42.atr',
 'I47.atr',
 'I10.dat',
 'I49.atr',
 'I55.dat',
 'I33.hea',
 'I24.atr',
 'I75.dat',
 'I51.atr',
 'I68.hea',
 'I14.dat',
 'I36.dat',
 'I34.atr',
 'I05.hea',
 'I03.hea',
 'I67.atr',
 'I25.hea',
 'I71.atr',
 'I01.atr',
 'I48.dat',
 'I06.dat',
 'I60.hea',
 'I24.dat',
 'I72.hea',
 'I18.atr',
 'I71.dat',
 'I17.atr',
 'I07.atr',
 'I31.atr',
 'I16.dat',
 'I53.atr',
 'I02.atr',
 'I38.hea',
 'I51.dat',
 'I12.dat',
 'I28.hea',
 'I02.hea',
 'I59.hea',
 'I73.hea',
 'I03.atr',
 'I35.hea',
 'I17.hea',
 'I57.dat',
 'I51.hea',
 'I44.dat',
 'I41.atr',
 'I24.hea',
 'I60.dat',
 'I70.dat',
 'I28.dat',
 'I04.hea',
 'I48.atr',
 'I30.dat',
 'I37.dat',
 'I52.dat',
 'I43.hea',
 'I59.atr',
 'I32.dat',
 'I09.hea',
 'I32.atr',
 'I45.hea',
 'I20.dat',
 'I04.atr',
 'I3

In [4]:
# Find the list of records to consider
rec_list = sorted(list(set([os.path.splitext(x)[0] for x in os.listdir(dl_dir)])))
print(rec_list)

['I01', 'I02', 'I03', 'I04', 'I05', 'I06', 'I07', 'I08', 'I09', 'I10', 'I11', 'I12', 'I13', 'I14', 'I15', 'I16', 'I17', 'I18', 'I19', 'I20', 'I21', 'I22', 'I23', 'I24', 'I25', 'I26', 'I27', 'I28', 'I29', 'I30', 'I31', 'I32', 'I33', 'I34', 'I35', 'I36', 'I37', 'I38', 'I39', 'I40', 'I41', 'I42', 'I43', 'I44', 'I45', 'I46', 'I47', 'I48', 'I49', 'I50', 'I51', 'I52', 'I53', 'I54', 'I55', 'I56', 'I57', 'I58', 'I59', 'I60', 'I61', 'I62', 'I63', 'I64', 'I65', 'I66', 'I67', 'I68', 'I69', 'I70', 'I71', 'I72', 'I73', 'I74', 'I75']


In [8]:
def patient_info(rec_name,comments):
  '''Classify patient disease:
  Coronary artery disease: 0
  MI: 1
  Transient ischemic attack: 2
  Other: 3
  
  Extract gender: M/F
  Extract age
  '''

  # Disease
  if 'Coronary artery disease' in comments[0]:
    disease = 0
  elif 'MI' in comments[0]:
    disease = 1
  elif 'Transient ischemic attack' in comments[0]:
    disease = 2
  else:
    disease = 3

  # Gender
  if '<sex>: M' in comments[0]:
    gender = 'M'
  else:
    gender = 'F'

  # Age
  age = int(fields['comments'][0][7:9])

  return disease,age,gender